# Day 17
Find the description of the problem [here](https://adventofcode.com/2024/day/17)!

## Part 1

Puzzle input:

In [1]:
with open("input_files/day_17.txt") as input_file:
    input = input_file.read()

Test input:

In [2]:
# # Comment this cell to use the puzzle input instead of the test input
# input = """Register A: 2024
# Register B: 0
# Register C: 0

# Program: 0,3,5,4,3,0"""

Parse the input:

In [3]:
import re

input_split = input.split("\n")

initial_A = int(re.findall(r"\d+",input_split[0])[0])
initial_B = int(re.findall(r"\d+",input_split[1])[0])
initial_C = int(re.findall(r"\d+",input_split[2])[0])
program = list(map(int, re.findall(r"\d+",input_split[4])))

Define all the possible operations:

In [4]:
# Some helper functions
def decimal_to_binary(number):
    return bin(number)[2:].zfill(3)

def binary_to_decimal(number_str):
    bits = list(map(int, list(number_str)))
    bits.reverse()
    number = 0
    for i, bit in enumerate(bits):
        number += bit * 2 ** i
    return number

# Calculate the combo operand
def combo(operand):
    if operand <= 3:
        return operand
    elif operand == 4:
        return A
    elif operand == 5:
        return B
    elif operand == 6:
        return C
    else:
        print("Invalid program")
        return -1

# Opcode 0:
def adv(operand):
    global A
    numerator = A
    denominator = 2 ** combo(operand)
    A = numerator // denominator

# Opcode 1:
def bxl(operand):
    global B
    binary_1 = decimal_to_binary(B)
    binary_2 = ""
    for _ in range(len(binary_1) - 3):
        binary_2 += "0"
    binary_2 += decimal_to_binary(operand)

    result = ""
    for bit_1, bit_2 in zip(binary_1, binary_2):
        result += "0" if bit_1 == bit_2 else "1"
    
    B = binary_to_decimal(result)
    

# Opcode 2:
def bst(operand):
    global B
    B = combo(operand) % 8

# Opcode 3:
def jnz(operand):
    global i
    global skip_increment
    if A == 0:
        return
    else:
        i = operand
        skip_increment = True

# Opcode 4:
def bxc(operand):
    global B
    binary_1 = decimal_to_binary(B)
    binary_2 = decimal_to_binary(C)
    len_1 = len(binary_1)
    len_2 = len(binary_2)
    if len_1 > len_2:
        binary_2 = ""
        for _ in range(len_1 - len_2):
            binary_2 += "0"
        binary_2 += decimal_to_binary(C)
    elif len_1 < len_2:
        binary_1 = ""
        for _ in range(len_2 - len_1):
            binary_1 += "0"
        binary_1 += decimal_to_binary(B)

    result = ""
    for bit_1, bit_2 in zip(binary_1, binary_2):
        result += "0" if bit_1 == bit_2 else "1"
    
    B = binary_to_decimal(result)

# Opcode 5:
def out(operand):
    global output
    output.append(combo(operand) % 8)

# Opcode 6:
def bdv(operand):
    global B
    numerator = A
    denominator = 2 ** combo(operand)
    B = numerator // denominator

# Opcode 7:
def cdv(operand):
    global C
    numerator = A
    denominator = 2 ** combo(operand)
    C = numerator // denominator

In [5]:
output = []
A = initial_A
B = initial_B
C = initial_C

i = 0
while i < len(program):
    skip_increment = False
    opcode = program[i]
    operand = program[i + 1]

    if opcode == 0:
        adv(operand)
    elif opcode == 1:
        bxl(operand)
    elif opcode == 2:
        bst(operand)
    elif opcode == 3:
        jnz(operand)
    elif opcode == 4:
        bxc(operand)
    elif opcode == 5:
        out(operand)
    elif opcode == 6:
        bdv(operand)
    elif opcode == 7:
        cdv(operand)
        
    if not skip_increment:
        i += 2

output_string = ",".join(list(map(str, output)))
print(f"The output is {output_string}")

The output is 2,0,4,2,7,0,1,0,3


## Part 2

Brute force doesn't work, as the number is very, very big.

In [6]:
# running = True
# counter = 0
# while running:
#     print(counter) if counter % 10000 == 0 else None

#     output = []
#     A = counter
#     B = initial_B
#     C = initial_C
    
#     i = 0
#     while i < len(program):
#         skip_increment = False
#         opcode = program[i]
#         operand = program[i + 1]

#         if opcode == 0:
#             adv(operand)
#         elif opcode == 1:
#             bxl(operand)
#         elif opcode == 2:
#             bst(operand)
#         elif opcode == 3:
#             jnz(operand)
#         elif opcode == 4:
#             bxc(operand)
#         elif opcode == 5:
#             out(operand)
#         elif opcode == 6:
#             bdv(operand)
#         elif opcode == 7:
#             cdv(operand)
            
#         for index, value in enumerate(output):
#             if output[index] != program[index]:
#                 break

#         if not skip_increment:
#             i += 2
    
#     if output == program:
#         break

#     counter += 1

# print(f"The lowest value of A to cause the program to output a copy of itself is {counter}.")

This method was reached after many many observations to the behaviour of changing the A value:

In [7]:
running = True

numbers = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]  # The n-th number represents n * 8 ** pos, where n is the value and pos is the index

# Iterate from the biggest number, which changes the right-most output value
position = len(numbers) - 1
while position >= 0:
    output = []
    A = 0
    for index, number in enumerate(numbers):
        A += number * 8 ** index
    B = initial_B
    C = initial_C

    i = 0
    while i < len(program):
        skip_increment = False
        opcode = program[i]
        operand = program[i + 1]

        if opcode == 0:
            adv(operand)
        elif opcode == 1:
            bxl(operand)
        elif opcode == 2:
            bst(operand)
        elif opcode == 3:
            jnz(operand)
        elif opcode == 4:
            bxc(operand)
        elif opcode == 5:
            out(operand)
        elif opcode == 6:
            bdv(operand)
        elif opcode == 7:
            cdv(operand)

        if not skip_increment:
            i += 2

    if output[position] == program[position]:
        # If the value coincides between the output and the program, go to the next number
        position -= 1
    elif numbers[position] >= 7 and position < len(numbers) - 1:
        # If it doesn't and we reach 7, go back to the previous number
        numbers[position] = 0
        position += 1
        numbers[position] += 1
    else:
        # If it doesn't but it's not 7 yet, just increase that number
        numbers[position] += 1

# Reconstruct A from the numbers
A = 0
for index, number in enumerate(numbers):
    A += number * 8 ** index
print(f"The lowest value of A to cause the program to output a copy of itself is {A}.")


The lowest value of A to cause the program to output a copy of itself is 265601188299675.
